#"Will It Learn?" - Shell Game
Will a simple convolution network learn a simple shell game? A circle will move from the center to one of four hiding places in the corners, obscured by an X. Then the shell with move radomly clockwise or anti-clockwise. Can it correctly give the hiding place?

###First we will create some functions to draw the world:

In [0]:
#This is a code cell. Click on the cell to make it active. 
#Then click on the left corner arrow in a circle button to run it. Or use the keyboard shortcut ctlr+enter.
import numpy as np
from PIL import Image, ImageDraw
import random
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline

#We will give a single tensor with each channel the board state
num_shuffles = 4
num_board_states = 3 + num_shuffles * 4
img_shape = (100, 100, num_board_states)

#our board state will use these symbols to represent the cup, the ball, and empty space
PLYR_X = 'X'
PLYR_O = 'O'
EMPTY = ' '

num_corners = 4
corner_names = ["top left", "top right", "bottom right", "bottom left"]

w = img_shape[1]
h = img_shape[0]

squares = []
for j in range(5):
  y = (h * j) / 5 + h / 10.0
  for i in range(5):
    x = (w * i) / 5 + w / 10.0
    squares.append((x, y))
    
def draw_x(draw, coord, sw, sh, color='white'):
  sz = sw
  x1 = coord[0] - sw / 2
  y1 = coord[1] - sh / 2
  x2 = x1 + sz
  y2 = y1 + sz
  coords = (x1, y1, x2, y2)
  draw.line(coords, fill=color)
  t = y1
  y1 = y2
  y2 = t
  coords = (x1, y1, x2, y2)
  draw.line(coords, fill=color)


def draw_o(draw, coord, sw, sh, color='white'):
  sz = sw
  x1 = coord[0] - sw / 2
  y1 = coord[1] - sh / 2
  x2 = x1 + sz
  y2 = y1 + sz
  coords = (x1, y1, x2, y2)
  draw.ellipse(coords, outline=color)  


def img_world(img_shape, board_state, squares):
  w = img_shape[1]
  h = img_shape[0]
  img = Image.new('RGB', size=(w, h))
  draw = ImageDraw.Draw(img)
  
  #draw field lines
  for i in range(4):
    horiz = h / 5.0 * (i + 1)
    draw.line((0, horiz, w, horiz), fill='white')

  for i in range(4):
    vert = w / 5.0 * (i + 1)
    draw.line((vert, 0, vert, h), fill='white')    
  
  sw = w / 10
  sh = h / 10
      
  for iS, state in enumerate(board_state):
    if state == PLYR_X:
      draw_x(draw, squares[iS], sw, sh)
    elif state == PLYR_O:
      draw_o(draw, squares[iS], sw, sh)
  
  return img
    

#test out world draw routine. Here's our start state:
board_state = [PLYR_X, EMPTY, EMPTY, EMPTY, PLYR_X,
               EMPTY, EMPTY, EMPTY, EMPTY, EMPTY,
               EMPTY, EMPTY, PLYR_O, EMPTY, EMPTY,
               EMPTY, EMPTY, EMPTY, EMPTY, EMPTY,
               PLYR_X, EMPTY, EMPTY, EMPTY, PLYR_X ]

img = img_world(img_shape, board_state, squares)
plt.imshow(img)
plt.show()

#This should show a grid of 25 squares with a circle in the center and x in four corners.

##Next we will create some routines to generate each board state of the game:

In [0]:
def start_board(corners, center):
  board_state = [EMPTY for i in range(25)]
  for corner in corners:
    board_state[corner] = PLYR_X
  p = PLYR_O
  board_state[center] = p          
  return board_state

def middle_board(corners, center):
  board_state = [EMPTY for i in range(25)]
  for corner in corners:
    board_state[corner] = PLYR_X
  possible_corners = [6, 8, 18, 16]
  iC = random.randint(0, 3)
  p = PLYR_O
  iS = possible_corners[iC]
  board_state[iS] = p
  return board_state, iC

def hidden_board(corners, center):
  board_state = [EMPTY for i in range(25)]
  for corner in corners:
    board_state[corner] = PLYR_X
  return board_state

def shuffled_board(corners):
  board_state = [EMPTY for i in range(25)]      
  for corner in corners:
    board_state[corner] = PLYR_X    
  return board_state
  
def get_board_imgs():
  '''
  generate a sequence of board images. The start state is always the circle
  in the middle with four possible corners. Then move the circle to one
  corner where it is hidden. Then slide the 'cups' closewise or counter-
  clockwise, or leave as is.
  '''
  corners = [0, 4, 24, 20]
  corners_pos_delta = [1, 5, -1, -5]
  corners_neg_delta = [5, -1, -5, 1]
  corners_zero_delta = [0, 0, 0, 0]
  center = 12
  

  start_state = start_board(corners, center)
  middle_state, iC = middle_board(corners, center)
  hidden_state = hidden_board(corners, center)

  img_s = img_world(img_shape, start_state, squares)
  img_s = np.array(img_s).astype(np.uint8)

  img_m = img_world(img_shape, middle_state, squares)
  img_m = np.array(img_m).astype(np.uint8)

  img_e = img_world(img_shape, hidden_state, squares)
  img_e = np.array(img_e).astype(np.uint8)
  
  imgs = [img_s, img_m, img_e]
  
  board_moved = hidden_state 
  
  
  for s in range(num_shuffles):
    
    #which dir will shells move, CW, CCW, or stationary (0)
    turn_dir = random.randint(-1, 1) #produces -1, 0, or, 1
    
    corners = [0, 4, 24, 20]
    if turn_dir == -1:
      delta = corners_neg_delta      
    elif turn_dir == 1:
      delta = corners_pos_delta
    else:
      delta = corners_zero_delta

    corners = [0, 4, 24, 20]

    for i in range(4):
      for j in range(4):
        corners[j] += delta[j]

      board_moved = shuffled_board(corners)
      img_t = img_world(img_shape, board_moved, squares)
      img_t = np.array(img_t).astype(np.uint8)  
      imgs.append(img_t)


    iC = iC + turn_dir
    iC = iC % 4
  
  return imgs, iC


def plot_boards(imgs, iN):
  plt.axis('off')
  f, axarr = plt.subplots(3, 4)
  f.set_figheight(5)
  f.set_figwidth(15)
  for i in range(3):
    for j in range(4):
      iImage = i * 4 + j
      axarr[(i, j)].axis('off')
      
      if iImage == len(imgs):
        break
      
      axarr[(i, j)].legend(['%d'% iImage])
      axarr[(i, j)].imshow(imgs[i * 4 + j])
      
  plt.show()  
  print("Circle is hiding in the", corner_names[iN])
  
  
def plot_anim(imgs, iN):   

  def update(i):
      img = imgs[i]
      ax.imshow(img)
      return ax
  
  label = "Circle is hiding in the " + corner_names[iN]
  fig, ax = plt.subplots()
  ax.axis('off')  
  ax.set_title(label)
  anim = FuncAnimation(fig, update, frames=np.arange(0, num_board_states), interval=200)
  return anim
  

#This will generate a sequence of board states.
#At the end, we have to predict where the circle is hiding. Pretty easy for humans.
#Less clear that a NN can learn to shift the literal circle into a virtual idea
#of an identicle visual X with additional info that it hides a circle.
imgs, iN = get_board_imgs()

anim = plot_anim(imgs, iN)
from IPython.display import HTML
HTML(anim.to_jshtml())
#sorry, this also displays the first frame in a seperate plot in the lower left. Not sure how to stop that.

#Hit the play triangle button on this to see an animation of the game.
#Run this code cell multiple times to see different random outcomes.

##What do you think?
Will this somewhat simple network accomplish the task of learning where the circle is hiding from just pixels? 


Let's find out...

First let's make sure you are using the GPU. Run the cell bellow to double check:

In [0]:
import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()
#if you see something like '/device:GPU:0' then the gpu accelerated learning will be enabled. if not, cpu will work, just slower.

##Our Neural Network

Here's what our network will look like. Pretty straight foward, 4 layers of convolutions. 64 filters per layer. A 3x3 kernel used in each. With relu activation in each. 

We will give it all the images planes stacked into one image. We will ask it to activate one of four neurons to represent which corner it thinks the circle is hidding at the end.


In [0]:
#Let's define a pretty straight forward conv neural network. It will have about 200K params, given the input image dimension of 100 x 100
import keras
from keras.layers import Flatten, Conv2D, Dense

img = keras.Input(shape=img_shape)

x = img
x = Conv2D(64, (3, 3), strides=(2,2), activation="relu")(x)
x = Conv2D(64, (3, 3), strides=(2,2), activation="relu")(x)
x = Conv2D(64, (3, 3), strides=(2,2), activation="relu")(x)
x = Conv2D(64, (3, 3), strides=(2,2), activation="relu")(x)
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dense(num_corners, activation='softmax')(x)

model = keras.Model(img, x)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
print(model.summary())

###Here's some rountines to stack our images into the channels of a single image tensor:

In [0]:
def rgb2gray(rgb):
  '''
  take a numpy rgb image return a new single channel image converted to greyscale
  '''
  return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def norm_img(img):
    return (img - img.mean() / np.std(img))/255.0
  
def stack_images(imgs, iC):
  width, height, _ = imgs[0].shape
  num_channels = len(imgs)

  img = np.zeros([width, height, num_channels], dtype=np.dtype('B'))

  for i, _img in enumerate(imgs):
    _img = rgb2gray(_img)
    img[...,i] = np.reshape(_img, (width, height))
  
  one_hot = np.zeros(4)
  one_hot[iC] = 1
  img = norm_img(np.float32(img))
  return img, one_hot
  
def get_data():
  '''
  generate training data. Get the images of the game sequence,
  convert them to greyscale, then stack them as channels in a single image
  '''
  imgs, iC = get_board_imgs()
  img, one_hot = stack_images(imgs, iC)
  return img, one_hot

def show_img_channels(img):
  '''
  show each channel of an image
  '''
  width, height, depth = img.shape
  plt.axis('off')
  for i in range(depth):
    plt.imshow((img[...,i].reshape(height, width) * 255).astype(np.uint8))
    plt.show()

#let's take a look at a sample of our training data:
img, one_hot = get_data()
print("answer:", one_hot)
show_img_channels(img)

#This should show about 15+ images that represent the states of the game.

###Now let's start the training:

In [0]:
# Here's function that will generate a numpy array of images and moves for our NN to train on
def generator(img_shape, batch_size, squares):
  while True:
    X = []
    y = []
    for i in range(batch_size):
      img, move = get_data()
      X.append(img)
      y.append(move)
    yield np.array(X), np.array(y)
    
    
#Now let's start the training.
batch_size = 64
steps_per_epoch = 128
epochs = 5

train_gen = generator(img_shape, batch_size, squares)

model.fit_generator(train_gen, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=epochs, 
                    verbose=1)


In [0]:
#Now let's create a new image and see how it does.
#Run this code cell multiple times to get a feeling about how it handles different cases.

imgs, iC = get_board_imgs()
img, move = stack_images(imgs, iC)
res = model.predict(img[None, :, :, :])
print("raw prediction: ", res[0])
iPred = np.argmax(res[0])
print("corner", iPred, "confidence", res[0][iPred])

print("Predicts circle is hiding in the", corner_names[iPred])
print("Answer:", corner_names[iC])

anim = plot_anim(imgs, iC)
HTML(anim.to_jshtml())


#So What's Your Verdict?
###Do you think this learned to predict where the circle was hiding?

In my opinion, it learned far better than expected. The fact that it seemed to confer additional state to a visually identical X and then track it's movement separate from others was quite surprising. 

It was interesting to see how it struggled to improve for the first few epochs. This seems to be near the limit of what this network can learn. As the weights are initializeded at random, this won't always succeed. If the accuracy doesn't improve over .3 then try training again.


###Note: 
this wasn't working until I normalized the channel images between 0-1

#Further thought..

This is still quite a simplified version of this game. Since there are only 324 possible board sequences to the game, perhaps it has simply memorized each possiblity. Perhaps it would struggle with additional shuffles of the board. Perhaps they could be shuffled diagonally as well.

###What tests could you devise to disambiguiate whether it had simply 'memorized' the world state?

Does this only work with all the board states at once? Can this be made to accept any length of sequences with sliding window on the complete state?



#Episodes Links:

[Will It Learn? - S01E01 Circle Count](https://drive.google.com/open?id=11EiFFa-imh5MNEPJZuqgqJAwLYHhP3gG)

[Will It Learn? - S01E02 Tic Tace Toe](https://drive.google.com/open?id=1PKosDR9wcgPaF2-BYMSZiu2nW03COxma)

[Will It Learn? - S01E03 : Shell Game](https://drive.google.com/open?id=163iv-LaidgxiU3tT_RcLCT_K1HOdagMu)

[Will It Learn? - S01E04 : Word Jumble](https://drive.google.com/open?id=19ENSHOC-TEyDqZ-_47QhSHHxhUAuDEoA)

[Will It Learn? - S01E05 : Mazes](https://drive.google.com/open?id=1qdYWNwrmYAtFsayzoxPuuGAE1RTKt1ia)


